# 7. Interactive visualization


In this module, were going to cover interactive visualization with [leafmap](https://leafmap.org/).

The end result will be similar to GIS tools like ArcGIS. But by having the visualization in python you get some big benefits compared to GIS

* Programatic nature means you can tweak your analaysis/data, and rebuild the visualization instantly.
* You get access to all python's tools, more than gis toolboxes.
* Because your viz is code, it can be repeated, version controlled, forked.

We'll cover vizualising each of the different "cardinalities" of geospatial data

* Points
* Lines
* Polygons
* Rasters


## Creating a map with leafmap

Leafmap is a python library that contains tools for creating an interactive map-based visualisation, as well as a wide variety of tools for downloading and processing geospatial data.

Most everything in leafmap starts by creating a new `Map` instance. To view the map in your notebook, we put just the variable of our map `m` on the last line of the cell: remember, notebooks auto-display their final line


In [ ]:
import leafmap

m = leafmap.Map()
m

By default, almost the entire world is shown. We can aim the map at our area of interest using the `center` and `zoom` arguments.

In [ ]:
latlon_carmel = (36.5506, -121.9220)
m = leafmap.Map(
    center=latlon_carmel,
    zoom=13,
)
m

Zooming can be disabled by setting `min_zoom` and `max_zoom`. Panning can be disabled with `dragging`. And leafmap's tools can be hidden with  the `clear_controls()` method.

Here's a static map:

In [ ]:
latlon_carmel = (36.5506, -121.9220)
zoom = 13
m = leafmap.Map(
    center=latlon_carmel,
    zoom=zoom,
    min_zoom=zoom,
    max_zoom=zoom,
    dragging=False,
)
m.clear_controls()
m

## Basemap selection

By default, leafmap uses an OpenStreetMap basemap. But for some applications you want other basemaps.

Leafmap comes with a number of basemaps, the latest list is [here](https://xyzservices.readthedocs.io/en/stable/gallery.html).

Some particularly useful ones are `Esri.WorldTopoMap` for terrain shading and contours

In [ ]:
m = leafmap.Map(
    center=latlon_carmel,
    zoom=13,
    basemap="Esri.WorldTopoMap",
)
m

`CartoDB.Positron` as a simple map that won't clash with your vizualisation overlays (there's also `CartoDB.PositronNoLabels` and `CartoDB.DarkMatter`, can you guess what they look like?)

In [ ]:
m = leafmap.Map(
    center=latlon_carmel,
    zoom=13,
    basemap="CartoDB.Positron",
)
m

`Esri.WorldImagery` has aerial imagery

In [ ]:
m = leafmap.Map(
    center=latlon_carmel,
    zoom=13,
    basemap="Esri.WorldImagery",
)
m

It can be useful to switch between base maps: for examply, you may want a simple layer for data visualisation, but also switch to aerial imagery to orient yourself.

Additional basemaps can be added with the `add_basemap()` method. The `add_layer_control()` method adds a popup to toggle layer visibility.

To keep our plots simple, we're also hiding the `draw` and `toolbar` controls here.

In [ ]:
m = leafmap.Map(
    center=latlon_carmel,
    zoom=13,
    basemap="Esri.WorldImagery",
    draw_control=False,
    toolbar_control=False,
)
m.add_basemap("CartoDB.PositronNoLabels")
m.add_layer_control()
m

## Loading data from SQL

Now we have our base map ready, lets load some data to visualise!

`data/USGSStreamgages.sqlite` is a database file that contains a table `SWIM_gage_loc` with [US streamgage information](https://www.sciencebase.gov/catalog/item/5ebe92af82ce476925e44b8f). 

For this course we're using a [sqlite](https://en.wikipedia.org/wiki/SQLite) database, as no server or authenticaion is needed: just a URL or file path. But the principles will be the same for other databases like Snowflake and PostGIS: you'll just provide your server hostname and credentials to connect instead.

The pandas function `read_sql_table()` loads all columns and rows from a database into a `DataFrame` object.





In [ ]:
import pandas as pd
from sqlalchemy import create_engine


# Connect to database.
#
# This will vary depending on the database being used.
db_connection_string = "sqlite:///data/USGSStreamgages.sqlite"
db_connection = create_engine(db_connection_string)

# Load the entire table.
#
# This will be the same for any SQL database, just provide a database connection object,
# pandas will handle the rest.
df_gage_full = pd.read_sql_table(table_name="SWIM_gage_loc", con=db_connection)

df_gage_full.head()

That's a lot of columns! A tip for viewing a dataframe that's so wide that has too many columns for the notebook to display them all, is to take the transpose of the first few rows.

In [ ]:
df_gage_full.head(2).transpose()

Lets extract just a subset of this data. We could of course use our pandas skills to filter out the relevant rows and columns! But this kind of simple filtering is exactly what SQL is designed for, and should load much faster as  we're not requesting unneeded data from the database.

Can you guess what our dataframe will look like after running this query?


In [ ]:
query = """
SELECT
    Gage_no,
    Gage_name,
    Lat_nwis,
    Long_nwis,
    Pass
FROM SWIM_gage_loc
WHERE State = 'CA'
"""

df_gage = pd.read_sql_query(query, db_connection)
df_gage.head()

Finally, because we have geospatial information in the dataframe, let's convert it from a `pandas` to a `geopandas` data frame, to give us access to the handy geospatial methods for processing the point locations.

In [ ]:
import geopandas as gpd

gdf_gage = gpd.GeoDataFrame(
    df_gage, geometry=gpd.points_from_xy(df_gage.Long_nwis, df_gage.Lat_nwis), crs="EPSG:4326"
)

gdf_gage.head()

## Displaying point data

Leafmap provides `add_markers_from_xy()` to add points from a dataframe.



In [ ]:
latlon_fresno = 36.857, -119.786
m = leafmap.Map(
    center=latlon_fresno,
    zoom=6,
    basemap="Esri.WorldTopoMap",
    draw_control=False,
    toolbar_control=False,
)

m.add_circle_markers_from_xy(
    data=gdf_gage,
)


m

Try clicking on one of the points: you get a popup showing the value of the other attributes of that streamgage.

With so many points, it's a bit hard to make sense of the data. Leafmap gives you [options](https://leafmap.org/leafmap/?h=add_circle_markers_from_xy#leafmap.leafmap.Map.add_circle_markers_from_xy) to style your markers: lets decrease the radius and opacity of the markers to get a better sense of the density.

As well as passing a constant value like `opacity=0.5`, we can also pass a list of values representing a different value for each marker. We'll also color-code the markers individually based on the "Pass" column of our dataframe (whether the streamgage data passed a QA check):


In [ ]:
latlon_fresno = 36.857, -119.786
m = leafmap.Map(
    center=latlon_fresno,
    zoom=6,
    basemap="Esri.WorldTopoMap",
    draw_control=False,
    toolbar_control=False,
)

color = ["blue" if passed else "gray" for passed in gdf_gage.Pass]

m.add_circle_markers_from_xy(
    data=gdf_gage,
    radius=2,  # Default is 10.
    opacity=0.5,  # Default is 1.
    color=color,
)

m



Reducing the marker size and opacity makes the map fairly readable. But for larger numbers, both visibility and browsing performance will start to suffer!

For even larger numbers of points, leafmap provides `add_marker_cluster()`. Each cluster displays how many points it includes: zooming in breaks up the clusters. 

In [ ]:
latlon_fresno = 36.857, -119.786
m = leafmap.Map(
    center=latlon_fresno,
    zoom=6,
    basemap="Esri.WorldTopoMap",
    draw_control=False,
    toolbar_control=False,
)

m.add_marker_cluster(
    data=gdf_gage,
    x="Long_nwis",  # Must provide the names of the x/y columns.
    y="Lat_nwis",
)

m

## Displaying vector data

Leafmap provides the `add_gdf()` function that can plot geopandas `GeoDataFrame` objects, including those with line and polygon geometry.

Here we load some streamlines around the SF bay area.

In [ ]:
gdf_streamlines = gpd.read_file("./data/California_Streams.SanFranciscoBay.geojson")

latlon_sf = 37.715, -122.475
m = leafmap.Map(
    center=latlon_sf,
    zoom=10,
    basemap="CartoDB.DarkMatter",
    draw_control=False,
    toolbar_control=False,
)

# Color the lines.
subbasin_colors = {
    "Tomales-Drake Bays": "blue",
    "San Francisco Coastal South": "purple",
}
style_callback = lambda feat: {"color": subbasin_colors[feat["properties"]["Hydrologic_Subbasin"]], "weight": 1}

# Plot the lines.
m.add_gdf(
    gdf_streamlines,
    style_callback=style_callback,
)

# Add a legend.
m.add_legend(title="Sub Basin", legend_dict=subbasin_colors)

m

## Displaying polygon data

Leafmap also comes with a number of helpful tools for downloading data.

The `get_wbd()` function returns watershed polygons based on a spatial query.

In [ ]:
m = leafmap.Map(
    center=latlon_carmel,
    zoom=12,
    basemap="Esri.WorldImagery",
    draw_control=False,
    toolbar_control=False,
)

point_geometry = {"y": latlon_carmel[0], "x": latlon_carmel[1]}
gdf_huc12 = leafmap.get_wbd(point_geometry, digit=12, return_geometry=True)
m.add_gdf(
    gdf_huc12,
)

m

Geopandas has the concept of a [spatial join](https://geopandas.org/en/stable/gallery/spatial_joins.html), using the `sjoin()` method. It lets you merge GeoDataFrame objects be finding rows whose geometry intersects.

For example: we can use a spatial join take our streamgage data frame, and for each gage row,  find the watershed that contains the point, and add the watershed attributes to the gage row.

In [ ]:
# Get HUC4 watershed boundaries for the area containing our streamgages.
xmin, ymin, xmax, ymax = gdf_gage.total_bounds
bbox_geometry = {"xmin": xmin, "ymin": ymin, "xmax": xmax, "ymax": ymax}
gdf_huc2 = leafmap.get_wbd(bbox_geometry, digit=2, return_geometry=True)

# Spatial join with gages.
gdf_gage_watershed = gdf_gage.sjoin(gdf_huc2, how="left", predicate="contains")

# Base map.
m = leafmap.Map(
    center=latlon_fresno,
    zoom=7,
    basemap="Esri.WorldImagery",
    draw_control=False,
    toolbar_control=False,
)

# Add the watershed boundaries as the 1st layer.
m.add_gdf(
    gdf_huc2,
    layer_name="watershed",
    style={"opacity": 0.5},
)

# Add our points on top.
m.add_circle_markers_from_xy(
    data=gdf_gage_watershed,
    color="magenta",
    layer_name="gage",
    radius=3,  # Default is 10.
    opacity=0.5,  # Default is 1.
)


m



Now the tooltip for each point contains not only the information about the gage, but also the attributes of the containing watershed.

## Displaying raster data

Raster data can be displayed with the `add_raster()` method. 

The [viridis](https://cran.r-project.org/web/packages/viridis/vignettes/intro-to-viridis.html) colormap uses hue (color) to communicate more information than grayscale, and was engineered to be easier to read by people with most form of color blindness and color deficiency.

In [ ]:
m = leafmap.Map(
    center=(37, -118),
    zoom=6,
    draw_control=False,
    toolbar_control=False,
)

m.add_raster("./data/dem_90m.tif", colormap="viridis")

m

## Exporting visualizations

Leafmap provides the `to_html()` method for saving maps to disk. The resulting html file can be shared to others and opened in their browser, even without having python installed.

In [ ]:
m = leafmap.Map(
    center=latlon_carmel,
    zoom=13,
    basemap="Esri.WorldTopoMap",
)
m.to_html("./tmp/exported_map_analysis.html")